# 🔬 **Multi-Magnification Breast Classification using VGG16 + Inception Block by performing Oversampling**
***

### 🔬 40X Magnification

- Dataset: 625 benign and 1370 malignant images.  
- Images are loaded from class-specific directories.  
- Stratified data split: 60% train, 10% val, 30% test.  
- Images resized to 224×224 and normalized.  
- VGG16 + Inception model with oversampling applied.



In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.utils import resample  # Added for oversampling

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/40X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/40X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

# ---------------------- OVERSAMPLING ----------------------
train_images = np.array(train_images)
train_labels = np.array(train_labels)

benign_mask = train_labels == 0
malignant_mask = train_labels == 1

benign_images_train = train_images[benign_mask]
malignant_images_train = train_images[malignant_mask]

benign_labels_train = train_labels[benign_mask]
malignant_labels_train = train_labels[malignant_mask]

# Oversample benign class to match malignant
benign_images_upsampled, benign_labels_upsampled = resample(
    benign_images_train,
    benign_labels_train,
    replace=True,
    n_samples=len(malignant_images_train),
    random_state=42
)

# Combine and shuffle
oversampled_train_images = np.concatenate([malignant_images_train, benign_images_upsampled])
oversampled_train_labels = np.concatenate([malignant_labels_train, benign_labels_upsampled])

shuffle_idx = np.random.permutation(len(oversampled_train_images))
oversampled_train_images = oversampled_train_images[shuffle_idx]
oversampled_train_labels = oversampled_train_labels[shuffle_idx]

# Dataset pipelines
train_dataset = tf.data.Dataset.from_tensor_slices((oversampled_train_images, oversampled_train_labels)) \
    .map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# ------------------ MODEL ------------------
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Add Inception block after VGG16
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ------------------ TRAIN ------------------
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# ------------------ EVALUATION ------------------
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 626
Total Malignant Images: 1370
Training samples: 1197
Validation samples: 200
Testing samples: 599
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 203s 8s/step - accuracy: 0.4977 - loss: 1.1253 - val_accuracy: 0.3150 - val_loss: 0.7155
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.5082 - loss: 0.6918 - val_accuracy: 0.7400 - val_loss: 0.6691
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.6772 - loss: 0.6733 - val_accuracy: 0.7750 - val_loss: 0.6439
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.6938 - loss: 0.6219 - val_accuracy: 0.7700 - val_loss: 0.5084
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.7388 - loss: 0.5503 - val_accuracy: 0.8050 - val_loss: 0.4619
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.7973 - loss: 0.4681 - val_accuracy: 0.8000 - val_loss: 0.4429
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0



### 🔬 100X Magnification

- Dataset: 644 benign and 1437 malignant images.  
- Loaded from benign/malignant directories.  
- Stratified train/val/test split maintained.  
- Images preprocessed (resize to 224×224 + normalization).  
- Model: VGG16 base + Inception block; oversampling enabled.


In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.utils import resample  # Required for oversampling

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/100X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/100X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

# ---------------------- OVERSAMPLING ----------------------
train_images = np.array(train_images)
train_labels = np.array(train_labels)

benign_mask = train_labels == 0
malignant_mask = train_labels == 1

benign_images_train = train_images[benign_mask]
malignant_images_train = train_images[malignant_mask]

benign_labels_train = train_labels[benign_mask]
malignant_labels_train = train_labels[malignant_mask]

# Upsample benign to match malignant
benign_images_upsampled, benign_labels_upsampled = resample(
    benign_images_train,
    benign_labels_train,
    replace=True,
    n_samples=len(malignant_images_train),
    random_state=42
)

# Combine and shuffle
oversampled_train_images = np.concatenate([malignant_images_train, benign_images_upsampled])
oversampled_train_labels = np.concatenate([malignant_labels_train, benign_labels_upsampled])

shuffle_idx = np.random.permutation(len(oversampled_train_images))
oversampled_train_images = oversampled_train_images[shuffle_idx]
oversampled_train_labels = oversampled_train_labels[shuffle_idx]

# Create dataset pipelines
train_dataset = tf.data.Dataset.from_tensor_slices((oversampled_train_images, oversampled_train_labels)) \
    .map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# ------------------ MODEL ------------------
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Add Inception block after VGG16
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ------------------ TRAIN ------------------
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# ------------------ EVALUATION ------------------
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 649
Total Malignant Images: 1437
Training samples: 1251
Validation samples: 209
Testing samples: 626
Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 140s 6s/step - accuracy: 0.4913 - loss: 1.0710 - val_accuracy: 0.3828 - val_loss: 0.7023
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 25s 987ms/step - accuracy: 0.5601 - loss: 0.6648 - val_accuracy: 0.7129 - val_loss: 0.6048
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 25s 995ms/step - accuracy: 0.7366 - loss: 0.5638 - val_accuracy: 0.8134 - val_loss: 0.4321
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.7957 - loss: 0.4651 - val_accuracy: 0.8230 - val_loss: 0.3923
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 38s 974ms/step - accuracy: 0.8435 - loss: 0.3647 - val_accuracy: 0.7895 - val_loss: 0.4805
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.8890 - loss: 0.2608 - val_accuracy: 0.8421 - val_loss: 0.4127
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.9350 - loss: 0.1813 - val_accuracy: 0.861

### 🔬 200X Magnification

- Dataset: 623 benign and 1390 malignant images.  
- Benign samples oversampled to balance class sizes.  
- Stratified splitting into train, val, and test sets.  
- Preprocessing includes resizing and normalization.  
- Architecture: VGG16 + Inception block.



In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.utils import resample

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/200X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/200X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

# ---------------------- OVERSAMPLING ----------------------
train_images = np.array(train_images)
train_labels = np.array(train_labels)

benign_mask = train_labels == 0
malignant_mask = train_labels == 1

benign_images_train = train_images[benign_mask]
malignant_images_train = train_images[malignant_mask]

benign_labels_train = train_labels[benign_mask]
malignant_labels_train = train_labels[malignant_mask]

# Upsample benign to match malignant
benign_images_upsampled, benign_labels_upsampled = resample(
    benign_images_train,
    benign_labels_train,
    replace=True,
    n_samples=len(malignant_images_train),
    random_state=42
)

# Combine and shuffle
oversampled_train_images = np.concatenate([malignant_images_train, benign_images_upsampled])
oversampled_train_labels = np.concatenate([malignant_labels_train, benign_labels_upsampled])

shuffle_idx = np.random.permutation(len(oversampled_train_images))
oversampled_train_images = oversampled_train_images[shuffle_idx]
oversampled_train_labels = oversampled_train_labels[shuffle_idx]

# Create dataset pipelines
train_dataset = tf.data.Dataset.from_tensor_slices((oversampled_train_images, oversampled_train_labels)) \
    .map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# ------------------ MODEL ------------------
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Add Inception block after VGG16
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ------------------ TRAIN ------------------
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# ------------------ EVALUATION ------------------
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 623
Total Malignant Images: 1390
Training samples: 1207
Validation samples: 202
Testing samples: 604
Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 215s 8s/step - accuracy: 0.5615 - loss: 1.0097 - val_accuracy: 0.7376 - val_loss: 0.6642
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.7050 - loss: 0.6472 - val_accuracy: 0.6188 - val_loss: 0.6436
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 41s 992ms/step - accuracy: 0.7239 - loss: 0.5488 - val_accuracy: 0.7376 - val_loss: 0.4997
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 40s 933ms/step - accuracy: 0.7093 - loss: 0.5470 - val_accuracy: 0.5495 - val_loss: 0.7075
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 26s 944ms/step - accuracy: 0.7503 - loss: 0.4749 - val_accuracy: 0.8267 - val_loss: 0.4074
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 39s 948ms/step - accuracy: 0.8566 - loss: 0.3374 - val_accuracy: 0.7574 - val_loss: 0.5169
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 41s 930ms/step - accuracy: 0.8617 - loss: 0.3227 - val_accuracy:

4/5 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   

5/5 ━━━━━━━━━━━━━━━━━━━━ 37s 8s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.8764 - loss: 0.7680
Test Accuracy: 0.8808
F1 Score: 0.9149
G-Mean: 0.8483
Informedness (IBA): 0.7035


### 🔬 400X Magnification

- Dataset: 588 benign and 1232 malignant images.  
- Oversampling balances benign class in training.  
- Data split: 60% train, 10% val, 30% test using stratification.  
- Images resized to 224×224, normalized.  
- Model: Frozen VGG16 base + custom Inception block.


In [3]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.utils import resample

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/400X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/400X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

# ---------------------- OVERSAMPLING ----------------------
train_images = np.array(train_images)
train_labels = np.array(train_labels)

benign_mask = train_labels == 0
malignant_mask = train_labels == 1

benign_images_train = train_images[benign_mask]
malignant_images_train = train_images[malignant_mask]

benign_labels_train = train_labels[benign_mask]
malignant_labels_train = train_labels[malignant_mask]

# Upsample benign to match malignant
benign_images_upsampled, benign_labels_upsampled = resample(
    benign_images_train,
    benign_labels_train,
    replace=True,
    n_samples=len(malignant_images_train),
    random_state=42
)

# Combine and shuffle
oversampled_train_images = np.concatenate([malignant_images_train, benign_images_upsampled])
oversampled_train_labels = np.concatenate([malignant_labels_train, benign_labels_upsampled])

shuffle_idx = np.random.permutation(len(oversampled_train_images))
oversampled_train_images = oversampled_train_images[shuffle_idx]
oversampled_train_labels = oversampled_train_labels[shuffle_idx]

# Create dataset pipelines
train_dataset = tf.data.Dataset.from_tensor_slices((oversampled_train_images, oversampled_train_labels)) \
    .map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# ------------------ MODEL ------------------
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Add Inception block after VGG16
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ------------------ TRAIN ------------------
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# ------------------ EVALUATION ------------------
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 588
Total Malignant Images: 1232
Training samples: 1091
Validation samples: 183
Testing samples: 546
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 273s 7s/step - accuracy: 0.4984 - loss: 1.1057 - val_accuracy: 0.6776 - val_loss: 0.6253
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 74s 965ms/step - accuracy: 0.5338 - loss: 0.6902 - val_accuracy: 0.7049 - val_loss: 0.6597
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 22s 924ms/step - accuracy: 0.6980 - loss: 0.6476 - val_accuracy: 0.7541 - val_loss: 0.5678
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 41s 916ms/step - accuracy: 0.7209 - loss: 0.5818 - val_accuracy: 0.7158 - val_loss: 0.5600
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.7660 - loss: 0.5034 - val_accuracy: 0.7760 - val_loss: 0.4286
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 22s 920ms/step - accuracy: 0.8104 - loss: 0.4198 - val_accuracy: 0.6995 - val_loss: 0.5877
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 871ms/ste

The new metric values for each magnification:

In [5]:
import pandas as pd

# Metrics from the last provided values
data = {
    'Magnification': ['40X', '100X', '200X', '400X'],
    'Test Accuracy': [0.9015, 0.8738, 0.8808, 0.8571],
    'F1 Score': [0.9287, 0.9093, 0.9149, 0.8963],
    'G-Mean': [0.8805, 0.8435, 0.8483, 0.8234],
    'Informedness (IBA)': [0.7641, 0.6932, 0.7035, 0.6551]
}

df = pd.DataFrame(data)

# Style the DataFrame for display
styled_df = df.style \
    .format({
        'Test Accuracy': '{:.4f}',
        'F1 Score': '{:.4f}',
        'G-Mean': '{:.4f}',
        'Informedness (IBA)': '{:.4f}'
    }) \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('background-color', '#000000'), ('color', 'white'), ('font-weight', 'bold')]
    }]) \
    .hide(axis='index')

styled_df


Magnification,Test Accuracy,F1 Score,G-Mean,Informedness (IBA)
40X,0.9015,0.9287,0.8805,0.7641
100X,0.8738,0.9093,0.8435,0.6932
200X,0.8808,0.9149,0.8483,0.7035
400X,0.8571,0.8963,0.8234,0.6551
